# Distance to neighbors

In [1]:
import pickle
from os.path import join
import numpy as np
from cupyx.scipy.spatial import distance
import cupy as cp
from pathlib import Path
import sys

# Add the main directory to sys.path to be able to import config
sys.path.append(str(Path.cwd().parent))
from config import ROOT_DIR
from utils.tools import compute_distances, argsort_chunked

# PARAMS
number_of_words = 5000
# END PARAMS

distance_metric = "euclidean"
distances_dtype = np.float16  # Precision of the distances
fit_dtype = (
    np.uint32
)  # Integer size sufficient to encode the number of words in the vocabularies

fasttext_data_folderpath = ROOT_DIR

## Distance to selected neighbors

In [3]:
x_rank = 1  # Rank of neighbor x
y_rank = 2  # Rank of neighbor y
z_rank = 101 # Rank of neighbor z

Load fasttext

In [4]:
# Load fasttext vocabulary and store it into suitable structures
with open(join(fasttext_data_folderpath, "wiki.en.pkl"), "rb") as f:
    fasttext = pickle.load(f)

vocab_embs = cp.array(list(fasttext.values()))  # Put on GPU
vocab_size = vocab_embs.shape[0]
del fasttext  # Save RAM

CPU times: user 1min 27s, sys: 23.3 s, total: 1min 50s
Wall time: 1min 50s


Take *sample_size* words and compute the distances against the entire vocabulary and rank their neighbors

In [5]:
words_ids = np.random.choice(vocab_size, size=number_of_words, replace=False)
words_embeddings = vocab_embs[words_ids]

# Step by step distance computation for the selected words
distances = compute_distances(
    words_embeddings, vocab_embs, distance_metric, dtype=distances_dtype
)

# For each word, get a sorted list of their neighbors.
word_neighbors = argsort_chunked(distances, fit_dtype)

# For each word, get a sorted list of the distances with the entire vocabulary.
# Instead of sorting again, benefit from word_neighbors.
sorted_distances = np.take_along_axis(distances, word_neighbors, axis=-1)

CPU times: user 18min 27s, sys: 36.1 s, total: 19min 3s
Wall time: 19min 3s


For each word, compute the distance to their *x*-th and *y*-th neighbor, and between these two.

In [6]:
# Distance to the neighbor x
distances_to_x = sorted_distances[:, x_rank]

# Distance to the neighbor y
distances_to_y = sorted_distances[:, y_rank]

# Distance to the neighbor z
distances_to_z = sorted_distances[:, z_rank]

# Gather the ids of x, y and z
x_neighbors = word_neighbors[:, x_rank : x_rank + 1]
y_neighbors = word_neighbors[:, y_rank : y_rank + 1]
z_neighbors = word_neighbors[:, z_rank : z_rank + 1]

x_and_y_neighbors = np.concatenate((x_neighbors, y_neighbors), axis=1)
x_and_z_neighbors = np.concatenate((x_neighbors, z_neighbors), axis=1)

# Compute the distance between x and y
distances_between_x_and_y = np.empty((number_of_words), dtype=distances_dtype)
distances_between_x_and_z = np.empty((number_of_words), dtype=distances_dtype)
for i in range(number_of_words):
    # Using cdist because cupyx.scipy.spatial.distance.euclidean has a bug https://github.com/cupy/cupy/issues/8288
    distances_between_x_and_y[i] = distance.cdist(
        vocab_embs[x_and_y_neighbors[i][0] : x_and_y_neighbors[i][0] + 1],
        vocab_embs[x_and_y_neighbors[i][1] : x_and_y_neighbors[i][1] + 1],
        distance_metric,
    ).item()

    distances_between_x_and_z[i] = distance.cdist(
        vocab_embs[x_and_z_neighbors[i][0] : x_and_z_neighbors[i][0] + 1],
        vocab_embs[x_and_z_neighbors[i][1] : x_and_z_neighbors[i][1] + 1],
        distance_metric,
    ).item()

CPU times: user 1.85 s, sys: 23.8 ms, total: 1.88 s
Wall time: 1.89 s


Print the values for the right-hand side of Equation 18 and Equation 19, used in Table 2 in the paper

In [8]:
print(f"Average half distance to neighbor {x_rank} = {distances_to_x.mean()/2}")
print(
    f"Average eq19 for neighbor {x_rank} and {y_rank} = {(((distances_to_y**2)-(distances_to_x**2))/(2*distances_between_x_and_y)).mean()}"
)
print(
    f"Average eq19 for neighbor {x_rank} and {z_rank} = {(((distances_to_z**2)-(distances_to_x**2))/(2*distances_between_x_and_z)).mean()}"
)

Average half distance to neighbor 1 = 1.4951171875
Average eq19 for neighbor 1 and 2 = 0.2374267578125
Average eq19 for neighbor 1 and 101 = 0.91162109375
